In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as SC
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from scipy.stats import levy
from sklearn.preprocessing import MinMaxScaler
from algorithms import simulated_annealing,GeomDecay, random_hill_climb
from neural import NetworkWeights,ContinuousOpt,gradient_descent
from activation import relu
from sklearn.metrics import accuracy_score
from datetime import datetime

In [61]:
def get_weights(weights):
    #sws = [np.reshape(weights[0:96],(24,4)),np.reshape(weights[96:100],4),np.reshape(weights[100:116],(4,4)),np.reshape(weights[116:120],4),np.reshape(weights[120:124],(4,1)),weights[124]]
    sws = [np.reshape(weights[0:96],(24,4)),np.reshape(weights[96:100],4),np.reshape(weights[100:116],(4,4)),[0.,0.,0.,0.],np.reshape(weights[116:120],(4,1)),[0.]]
    return sws

def predict(weights,X_test,Y_test,model):
    sws = get_weights(weights)
    model.set_weights(sws)
    y_pred = model.predict(X_test)
    # setting a confidence threshhold of 0.9
    y_pred_labels = list(y_pred > 0.9)

    for i in range(len(y_pred_labels)):
        if int(y_pred_labels[i]) == 1 : y_pred_labels[i] = 1
        else : y_pred_labels[i] = 0
    return accuracy_score(Y_test,y_pred_labels)

def BackPropagation():
    model = Sequential()
    model.add(Dense(4,input_dim=X_train.shape[1]))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model1 = model
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(X_train,Y_train,batch_size=2,epochs=50,validation_data = (X_test,Y_test))
    y_pred = model.predict(X_test)
    # setting a confidence threshhold of 0.9
    y_pred_labels = list(y_pred > 0.9)

    for i in range(len(y_pred_labels)):
        if int(y_pred_labels[i]) == 1 : y_pred_labels[i] = 1
        else : y_pred_labels[i] = 0
    return accuracy_score(Y_test,y_pred_labels)


def EagleStrategyWithSA(model,iters):
    mn = MinMaxScaler(feature_range=(-3,3))
    r = levy.rvs(size=iters)
    max_accuracy = 0
    final_weights = []
    count = 1
    for i in list(r):
        weights = np.random.uniform(-1,1,120)
        weights = weights * i
        weights = mn.fit_transform(weights.reshape(-1,1))
        acc0 = predict(weights,X_test,Y_test,model)
        if acc0 > 0.5:

            clip_max = 3
            fitness = NetworkWeights(X_train,Y_train,[25,4,4,1],relu,True,True,0.2)
            problem = ContinuousOpt(120,fitness,maximize=False,min_val=-1*clip_max,max_val=clip_max,step=0.2)
            #print(problem.get_length(),len(weights))
            fitted_weights, loss = simulated_annealing(problem,schedule=GeomDecay(),max_attempts=50,max_iters=500,init_state=weights,curve=False)
            acc = predict(fitted_weights,X_test,Y_test,model)
            if acc > max_accuracy:
                #print(acc0)
                max_accuracy = acc
                final_weights = fitted_weights
            if max_accuracy > 0.99:
                break
            count+=1
    return max_accuracy,loss,count,final_weights  

def EagleStrategyWithHill(model,iters,rate):
    mn = MinMaxScaler(feature_range=(-3,3))
    r = levy.rvs(size=iters)
    max_accuracy = 0
    final_weights = []
    count = 1
    for i in list(r):
        weights = np.random.uniform(-1,1,120)
        weights = weights * i
        weights = mn.fit_transform(weights.reshape(-1,1))
        acc = predict(weights,X_test,Y_test,model)
        if acc > 0.5:
            clip_max = 5
            fitness = NetworkWeights(X_train,Y_train,[25,4,4,1],relu,True,True,rate)
            problem = ContinuousOpt(120,fitness,maximize=False,min_val=-1*clip_max,max_val=clip_max,step=rate)
            #print(problem.get_length(),len(weights))
            fitted_weights, loss = random_hill_climb(problem, max_attempts=10, max_iters=1000, restarts=0,
                      init_state=weights, curve=False, random_state=None)
            acc = predict(fitted_weights,X_test,Y_test,model)
            if acc > max_accuracy:
                max_accuracy = acc
                final_weights = fitted_weights
            if max_accuracy > 0.99:
                break
            count+=1
    return max_accuracy 

In [58]:
df = pd.read_csv('Eyes.csv')
X = df[df.columns[1:25]]
Y = df[df.columns[25]]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.10,random_state=42)

sc = SC()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_train, X_test, Y_train, Y_test = np.array(X_train), np.array(X_test), np.array(Y_train), np.array(Y_test)

In [59]:
# Create model
model = Sequential()
model.add(Dense(4,input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [62]:
start = datetime.now()
accuracy,loss,count,final_weights = EagleStrategyWithSA(model,40)
print("Time taken ",datetime.now()-start)
print(accuracy)
start = datetime.now()
accuracy = EagleStrategyWithHill(model,40,0.2)
print("Time taken ",datetime.now()-start)
print(accuracy)
start = datetime.now()
accuracy = BackPropagation()
print("Time taken ",datetime.now()-start)
print(accuracy)

Time taken  0:00:07.899885
0.96
Time taken  0:00:02.959091
1.0
Train on 450 samples, validate on 50 samples
Epoch 1/50
450/450 [==============================] - 1s 2ms/step - loss: 0.6317 - acc: 0.7222 - val_loss: 0.5022 - val_acc: 0.8000
Epoch 2/50
450/450 [==============================] - 0s 434us/step - loss: 0.4037 - acc: 0.7222 - val_loss: 0.3107 - val_acc: 0.8000
Epoch 3/50
450/450 [==============================] - 0s 417us/step - loss: 0.2998 - acc: 0.8378 - val_loss: 0.2507 - val_acc: 0.9400
Epoch 4/50
450/450 [==============================] - 0s 410us/step - loss: 0.2252 - acc: 0.9644 - val_loss: 0.2029 - val_acc: 0.9400
Epoch 5/50
450/450 [==============================] - 0s 412us/step - loss: 0.1397 - acc: 0.9733 - val_loss: 0.1383 - val_acc: 1.0000
Epoch 6/50
450/450 [==============================] - 0s 419us/step - loss: 0.0615 - acc: 1.0000 - val_loss: 0.0874 - val_acc: 1.0000
Epoch 7/50
450/450 [==============================] - 0s 426us/step - loss: 0.0307 - acc: 